# **HOW TO GENERATE A DATASET ABOUT COOKING ?**

## **CREATE A DATASET WITH OPEN-SOURCE MODEL : Phi-2**

**Why this method ?**

Using Phi-2 for dataset generation is a strategic choice for our student project, given our limited resources and small team of three. As students, we lack the financial means to access large-scale AI infrastructure, so we must find a balance between quantity and quality to ensure we have enough data to fine-tune our model effectively. Phi-2, being a lightweight yet capable language model, allows us to generate a high volume of structured instructional data without requiring access to expensive high-end GPUs. By carefully designing prompts and filtering the generated output, we aim to maintain quality while maximizing the number of examples, ensuring that our model is trained on a dataset that is both diverse and relevant. This approach enables us to push the limits of what can be achieved with minimal resources, making AI research accessible even to small student-led initiatives like ours.


**Why using a GPU ?**

We used a GPU to speed up the dataset generation process. Since Phi-2 is a Transformer-based model, running it on a CPU would be significantly slower, making it impractical to generate a large dataset efficiently. The GPU allows for faster text generation, reducing waiting times and enabling us to produce more data in less time, which is crucial given our limited resources and need for scalability.

In [1]:
import json
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer


c:\Users\lucie\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.1


**Let's initialize Phi-2 :**

To initialize Phi-2, we first load the tokenizer and model using Hugging Face's ```AutoTokenizer``` and ```AutoModelForCausalLM```. We specify ```torch.float16``` to reduce memory usage and improve efficiency, which is crucial for running the model smoothly. The ```device_map="auto"``` setting automatically assigns the model to the available hardware, utilizing the GPU if present for faster processing. This setup ensures that Phi-2 is optimized for inference while keeping resource consumption manageable

In [3]:
# Charger Phi-2
model_name = "phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]
Some weights of the model checkpoint at phi-2 were not used when initializing PhiForCausalLM: {'lm_head.bias'}
- This IS expected if you are initializing PhiForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing PhiForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## **QUESTION GENERATION :**

To generate questions and imperative sentences, we first create a JSON file containing structured prompts. Each prompt serves as a template that guides the model in producing new variations. The prompts are stored in V2_prompts_cuisine.json and are loaded into a Python variable for processing.

In [4]:
question_generation = True

In [5]:
# Chargement du fichier JSON dans une variable
with open("V2_prompts_cuisine.json", "r", encoding="utf-8") as f:
    prompt_list = json.load(f)

# Affichage des 4 premiers éléments pour vérifier
print(prompt_list[:4])

[{'indice': 1, 'prompt': 'Instruct: Generate a question that a beginner cook might ask.\nOutput:'}, {'indice': 2, 'prompt': 'Instruct: Provide a question someone might ask when learning how to bake bread.\nOutput:'}, {'indice': 3, 'prompt': 'Instruct: Generate a question someone might ask when learning to cook.\nOutput:'}, {'indice': 4, 'prompt': 'Instruct: Give an imperative cooking instructions a recipe might include.\nOutput:'}]


Once the prompts are loaded, we iterate through them and generate multiple variations for each, ensuring diversity in the dataset. The generation process uses Phi-2 to create unique questions while maintaining relevance. To optimize efficiency, we implement a save interval, periodically storing the dataset to prevent data loss. Additionally, a set of unique questions ensures that we avoid duplicates, improving dataset quality. This approach allows us to efficiently build a large and diverse dataset.

In [ ]:
# Nombre de questions à générer par prompt
num_questions_per_prompt = 15 # Modifier selon tes besoins
save_interval = 5  # Sauvegarde toutes les X questions
dataset = []
unique_questions = set()
total_questions = len(prompt_list) * num_questions_per_prompt

question_count = 0  # Compteur global

In [7]:
if question_generation:
    context_prompt = (
        "You are generating questions or instructions that a language model can understand and respond to. "
        "Ensure that the tasks involve text-based guidance, explanations, or structured information. "
        "Avoid instructions that require physical actions or interactions with the real world."
    )

    for prompt_entry in prompt_list:
        prompt_text = f"{context_prompt}\n{prompt_entry['prompt']}"

        for _ in range(num_questions_per_prompt):
            print(f"Generating question {question_count+1}/{total_questions} using prompt: {prompt_entry['indice']}...")

            attempts = 0
            max_attempts = 5  # Limite d'essais pour éviter les doublons

            while attempts < max_attempts:
                # Transformer le prompt en tokens
                inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
                prompt_length = inputs["input_ids"].shape[1]

                # Génération de texte
                output = model.generate(
                    inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_new_tokens=30,
                    do_sample=True,
                    temperature=0.75,
                    top_k=60,
                    pad_token_id=tokenizer.eos_token_id
                )

                # Décodage et extraction de la réponse
                generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

                # Nettoyage du texte généré
                if "Output:" in generated_text:
                    generated_text = generated_text.split("Output:")[1].strip()

                # Extraction et nettoyage de la question
                generated_lines = generated_text.split("\n")
                cleaned_questions = [q.strip() for q in generated_lines if q.strip()]

                # On prend uniquement la première ligne générée
                question = cleaned_questions[0] if cleaned_questions else "Unknown question"

                # Vérification si la question est unique
                if question not in unique_questions:
                    unique_questions.add(question)
                    dataset.append({"instruction": question, "response": ""})
                    question_count += 1
                    break  # Sort de la boucle si la question est unique

                attempts += 1

            if attempts == max_attempts:
                print(f"⚠️ Impossible de générer une question unique après {max_attempts} essais.")

            # Sauvegarde temporaire à intervalles réguliers
            if question_count % save_interval == 0:
                print(f"🔹 Saving at {question_count} questions...")
                df = pd.DataFrame(dataset)
                df.to_json("cooking_prompts_temp.json", orient="records", indent=4)

    # Sauvegarde finale
    df = pd.DataFrame(dataset)
    df.to_json("cooking_prompts_final.json", orient="records", indent=4)
    print("✅ Finished generating questions. Data saved successfully.")


Generating question 1/1170 using prompt: 1...
Generating question 2/1170 using prompt: 1...
Generating question 3/1170 using prompt: 1...
Generating question 4/1170 using prompt: 1...
Generating question 5/1170 using prompt: 1...
🔹 Saving at 5 questions...
Generating question 6/1170 using prompt: 1...
Generating question 7/1170 using prompt: 1...
Generating question 8/1170 using prompt: 1...
Generating question 9/1170 using prompt: 1...
Generating question 10/1170 using prompt: 1...
🔹 Saving at 10 questions...
Generating question 11/1170 using prompt: 1...
Generating question 12/1170 using prompt: 1...
Generating question 13/1170 using prompt: 1...
Generating question 14/1170 using prompt: 1...
Generating question 15/1170 using prompt: 1...
🔹 Saving at 15 questions...
Generating question 16/1170 using prompt: 2...
Generating question 17/1170 using prompt: 2...
Generating question 18/1170 using prompt: 2...
Generating question 19/1170 using prompt: 2...
Generating question 20/1170 using

## **ANSWER GENERATION :**

For the response generation process, we first load the previously created JSON file containing prompts. To avoid overwriting the original data, we create a deep copy of the dataset, ensuring that responses are added without modifying the initial prompts.

In [37]:
# Charger le fichier JSON existant
input_file = "V2_cooking_prompts_final.json"
output_file = "V2_cooking_prompts_with_responses.json"

In [38]:
import copy

with open(input_file, "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Créer une copie pour éviter d'écraser les données
dataset_with_responses = copy.deepcopy(dataset)

To guide Phi-2 in producing relevant and structured answers, we define a global prompt that sets a clear instruction framework. The model is instructed to act as a professional kitchen assistant, providing concise, practical, and relevant responses. The focus is on short, structured answers, avoiding unnecessary details while ensuring accuracy. When step-by-step instructions are required, they are kept brief and to the point, maintaining clarity and usefulness.

In [4]:
# Prompt global pour guider le modèle
global_prompt = (
    "Instruct: Act as a professional kitchen assistant and address the user as if they were a chef. "
    "Provide clear, concise, and practical answers to cooking-related questions. "
    "Keep your responses short and to the point, focusing only on essential information. "
    "Use professional yet accessible language, and avoid unnecessary details. "
    "If a step-by-step explanation is needed, limit it to a few key steps. "
    "Ensure accuracy and relevance, keeping answers within a few sentences. "
    "Now, answer the next question concisely:\n\n"
)


By following this structured approach, we ensure that the dataset contains high-quality responses that align with professional culinary guidance. This method allows for the creation of a coherent and instructive dataset.

In [40]:
# Boucle de génération des réponses
for i, entry in enumerate(dataset_with_responses):
    question = entry["instruction"]
    print(f"Generating response for question {i+1}/{len(dataset_with_responses)}: {question}")

    # Construire le prompt avec la question
    full_prompt = global_prompt + question + "\nOutput:\n"

    # Transformer le prompt en tokens
    inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
    
    # Génération de texte
    output = model.generate(
        inputs["input_ids"], 
        attention_mask=inputs["attention_mask"],
        max_new_tokens=300,
        do_sample=True, 
        temperature=0.7,  # Ajustement de la diversité
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )

    # Décoder et extraire la réponse
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    if "Output:" in generated_text:
                    generated_text = generated_text.split("Output:")[1].strip()

    # Ajouter la réponse dans le dataset
    entry["response"] = generated_text

    # Sauvegarde intermédiaire toutes les 10 réponses
    if (i + 1) % 10 == 0:
        print(f"🔹 Saving at {i+1} responses...")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(dataset_with_responses, f, indent=4, ensure_ascii=False)

Generating response for question 1/1167: What are the essential ingredients needed to make spaghetti carbonara?
Generating response for question 2/1167: How do I properly season a steak for grilling?
Generating response for question 3/1167: How do I season a cast iron skillet properly?
Generating response for question 4/1167: What is the best way to season a steak before grilling it?
Generating response for question 5/1167: What is the difference between baking powder and baking soda in a recipe?
Generating response for question 6/1167: What is the difference between baking soda and baking powder?
Generating response for question 7/1167: A beginner cook might ask, "How do I properly chop vegetables for a stir-fry dish?"
Generating response for question 8/1167: A beginner cook might ask, "How can I make my pasta sauce more flavorful?"
Generating response for question 9/1167: What is the best way to prepare a roast chicken before roasting it in the oven?
Generating response for question 

In [41]:
# Sauvegarde finale
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(dataset_with_responses, f, indent=4, ensure_ascii=False)

print("✅ Finished generating responses. Data saved successfully.")

✅ Finished generating responses. Data saved successfully.


In [9]:
# Fichiers d'entrée et de sortie
input_file = "V5_cooking_prompts_final_fixed.json"
output_file = "V6_cooking_prompts_final_fixed.json"

# Charger le dataset existant
with open(input_file, "r", encoding="utf-8") as f:
    dataset_with_responses = json.load(f)

In [10]:
# Repasser sur les entrées vides et regénérer
missing_responses = 0

for entry in dataset_with_responses:
    if entry["response"] == "":
        missing_responses += 1
        question = entry["instruction"]
        print(f"Generating response for question {missing_responses+1}/{len(dataset_with_responses)}: {question}")

        # Construire le prompt avec la question
        full_prompt = global_prompt + question + "\nOutput:\n"

        # Transformer le prompt en tokens
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        
        # Génération de texte
        output = model.generate(
            inputs["input_ids"], 
            attention_mask=inputs["attention_mask"],
            max_new_tokens=300,
            do_sample=True, 
            temperature=0.9,  # Ajustement de la diversité
            top_k=60,
            pad_token_id=tokenizer.eos_token_id
        )

        # Décoder et extraire la réponse
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()
        if "Output:" in generated_text:
                        generated_text = generated_text.split("Output:")[1].strip()

        print(generated_text)          
        entry["response"] = generated_text

# Sauvegarde du dataset mis à jour
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(dataset_with_responses, f, indent=4, ensure_ascii=False)

print(f"✅ Process completed. {missing_responses} missing responses were regenerated.")


Generating response for question 2/1167: Question: How do I properly season a steak before cooking it?
1. Start by rinsing the steak under cold water and patting it dry with paper towels.
2. Season both sides of the steak generously with salt and freshly ground black pepper.
3. Wait a few minutes to allow the salt to draw out any excess moisture from the meat.
4. Optional: Add additional seasonings such as garlic powder, onion powder, dried thyme, or your favorite spices to enhance the flavor. 
5. Make sure to pat the steak dry again before proceeding with your cooking method.
Generating response for question 3/1167: Question: How do I properly julienne carrots for a stir-fry?

Generating response for question 4/1167: What are some basic cooking techniques that beginners should learn before attempting more complex recipes?

Generating response for question 5/1167: The main distinction between frying and grilling lies in the cooking method and the resulting flavors. Frying involves subm